<a href="https://colab.research.google.com/github/gencross/NewtonGroup1/blob/main/Main_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello, world!")

Hello, world!
